In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


## EDA Inicial

In [2]:
# Ruta del archivo
ruta = r"C:\Users\marta\Desktop\mibootcamp\ETL_IBIShotels\data\reservas_hoteles.parquet"

# Leer el archivo Parquet con pandas
df_raw = pd.read_parquet(ruta, engine="pyarrow")

# Ignorar el índice del DataFrame
df_raw = df_raw.reset_index(drop=True)

df_raw.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,,2025-03-01,2025-03-02,113,NaN,,NaN,
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,,2025-03-01,2025-03-02,194,NaN,,NaN,
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid


In [3]:
df = df_raw.copy()
df = pd.DataFrame(df)
df.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,,2025-03-01,2025-03-02,113,NaN,,NaN,
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,,2025-03-01,2025-03-02,194,NaN,,NaN,
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_reserva       15098 non-null  object 
 1   id_cliente       15098 non-null  object 
 2   nombre           15098 non-null  object 
 3   apellido         15098 non-null  object 
 4   mail             15098 non-null  object 
 5   competencia      15098 non-null  bool   
 6   fecha_reserva    15098 non-null  object 
 7   inicio_estancia  15023 non-null  object 
 8   final_estancia   15023 non-null  object 
 9   id_hotel         15098 non-null  int64  
 10  precio_noche     9874 non-null   float64
 11  nombre_hotel     15098 non-null  object 
 12  estrellas        9926 non-null   float64
 13  ciudad           15098 non-null  object 
dtypes: bool(1), float64(2), int64(1), object(10)
memory usage: 1.5+ MB


In [5]:
df_col_fecha = df[["fecha_reserva", "inicio_estancia", "final_estancia"]]
df_col_fecha

,fecha_reserva,inicio_estancia,final_estancia
0,,2025-03-01,2025-03-02
1,,2025-03-01,2025-03-02
2,2025-02-09,2025-03-01,2025-03-02
3,2025-02-08,2025-03-01,2025-03-02
4,2025-02-02,2025-03-01,2025-03-02
...,...,...,...
15093,2025-02-02,2025-03-01,2025-03-02
15094,2025-02-07,2025-03-01,2025-03-02
15095,2025-02-08,2025-03-01,2025-03-02
15096,2025-02-10,2025-03-01,2025-03-02


Transformación inicial 

- transformación de co.fecha_reserva, incio_estancia, final_etancia check
- hay nulos en las columnas: inicio_estancia, final_instancia, precio_noche, nombre_hotel, estrellas.
- analizar duplicados
- asegurarse del tipo de dato es el correcto

In [6]:
def convertir_a_fecha(lista_columnas, dataframe):
    for col in lista_columnas:
        dataframe[col] = pd.to_datetime(dataframe[col], errors='coerce')
    return dataframe

In [7]:
col_fechas = ['inicio_estancia', 'final_estancia', 'fecha_reserva']
df = convertir_a_fecha(col_fechas, df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15098 non-null  object        
 1   id_cliente       15098 non-null  object        
 2   nombre           15098 non-null  object        
 3   apellido         15098 non-null  object        
 4   mail             15098 non-null  object        
 5   competencia      15098 non-null  bool          
 6   fecha_reserva    9926 non-null   datetime64[ns]
 7   inicio_estancia  15023 non-null  datetime64[ns]
 8   final_estancia   15023 non-null  datetime64[ns]
 9   id_hotel         15098 non-null  int64         
 10  precio_noche     9874 non-null   float64       
 11  nombre_hotel     15098 non-null  object        
 12  estrellas        9926 non-null   float64       
 13  ciudad           15098 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64

In [8]:
df.select_dtypes(include=['object']).columns  # Columnas categóricas

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail',
       'nombre_hotel', 'ciudad'],
      dtype='object')

In [9]:
df.select_dtypes(include=['number']).columns  # Columnas numéricas

Index(['id_hotel', 'precio_noche', 'estrellas'], dtype='object')

opción 2. para covertir a fecha: NO FUNCIONA

In [10]:
##visto en clase1
def conver_fecha(dataframe):
    for col in dataframe.columns:
        if "fecha" in col:
            dataframe[col] = pd.to_datatime(dataframe[col])
    return dataframe

In [11]:
def convertir_fecha(dataframe):
    for col in dataframe.filter(like="fecha", axis=1):
        dataframe[col] = pd.to_datetime(dataframe[col], errors='coerce')  # Forzar conversión
    return dataframe

# Aplicar la conversión a todas las columnas que contengan "fecha"
df = convertir_fecha(df)

# Verificar tipos de datos
df.info()                                    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15098 non-null  object        
 1   id_cliente       15098 non-null  object        
 2   nombre           15098 non-null  object        
 3   apellido         15098 non-null  object        
 4   mail             15098 non-null  object        
 5   competencia      15098 non-null  bool          
 6   fecha_reserva    9926 non-null   datetime64[ns]
 7   inicio_estancia  15023 non-null  datetime64[ns]
 8   final_estancia   15023 non-null  datetime64[ns]
 9   id_hotel         15098 non-null  int64         
 10  precio_noche     9874 non-null   float64       
 11  nombre_hotel     15098 non-null  object        
 12  estrellas        9926 non-null   float64       
 13  ciudad           15098 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64

In [12]:
df.describe().T 

,count,mean,min,25%,50%,75%,max,std
fecha_reserva,9926,2025-02-06 10:27:43.993552384,2025-02-01 00:00:00,2025-02-03 00:00:00,2025-02-06 00:00:00,2025-02-09 00:00:00,2025-02-12 00:00:00,NaN
inicio_estancia,15023,2025-02-28 23:59:59.999999744,2025-03-01 00:00:00,2025-03-01 00:00:00,2025-03-01 00:00:00,2025-03-01 00:00:00,2025-03-01 00:00:00,NaN
final_estancia,15023,2025-03-02 00:00:00,2025-03-02 00:00:00,2025-03-02 00:00:00,2025-03-02 00:00:00,2025-03-02 00:00:00,2025-03-02 00:00:00,NaN
id_hotel,15098.0,65.950192,2.0,27.0,40.0,114.0,194.0,57.832033
precio_noche,9874.0,275.11772,50.01,163.12,273.315,387.83,499.82,129.405972
estrellas,9926.0,3.012694,1.0,2.0,3.0,4.0,5.0,1.412446


## Incosistencia en los datos:

In [13]:
df["nombre_hotel"].nunique()

20

In [14]:
df["id_hotel"].nunique()

29

In [15]:
df['id_cliente'].nunique()


14847

In [16]:
df["id_reserva"].nunique()

15000

- debería haber los mismos nombres de hoteles que de id_hotel: hay más id_hotel que nombre:hotel por lo que puede haber un error al asignar los ID
- debería hacer los mismos id_clientes que id_reserva: hay más reservas que clientes, por lo que hay clientes que han hecho más de una reserva lo que en nuestro contexto no tiene sentido ya que los datos son para un mismo fin de semana. 

1️⃣ Analizando inconsistencias en ID de hoteles y nombres

In [32]:
df_hotel_check = df.groupby("nombre_hotel")["id_hotel"].nunique().reset_index()
df_hotel_check

,nombre_hotel,id_hotel
0,,10
1,Gran Hotel Madrid,19
2,Hotel Brisas del Mar,19
3,Hotel Camino del Sol,19
4,Hotel Costa Azul,19
5,Hotel Encanto Real,19
6,Hotel Jardines del Rey,19
7,Hotel Las Estrellas,19
8,Hotel Los Almendros,19
9,Hotel Luz de Madrid,19


In [18]:
df_hotel_check = df_hotel_check.sort_values(by="id_hotel", ascending=False)
print(df_hotel_check)

              nombre_hotel  id_hotel
10    Hotel Maravilla Real        19
1        Gran Hotel Madrid        19
18      Hotel Vista Alegre        19
17      Hotel Torre Dorada        19
16        Hotel Sol y Luna        19
15     Hotel Rincón Sereno        19
14  Hotel Puerta del Cielo        19
13  Hotel Palacio Imperial        19
12       Hotel Monte Verde        19
11      Hotel Mirador Real        19
19         Palacio del Sol        19
9      Hotel Luz de Madrid        19
8      Hotel Los Almendros        19
7      Hotel Las Estrellas        19
6   Hotel Jardines del Rey        19
5       Hotel Encanto Real        19
4         Hotel Costa Azul        19
3     Hotel Camino del Sol        19
2     Hotel Brisas del Mar        19
0                                 10


- Hay un conjunto de 10 hoteles sin nombre.
- Cada nombre de hotel tiene 19 ID's distintos.
- otro problema: hay id_cliente mal asignados (?)

In [33]:
df[["nombre_hotel", "id_hotel"]].drop_duplicates().groupby("nombre_hotel")["id_hotel"].count()


nombre_hotel
                          10
Gran Hotel Madrid         19
Hotel Brisas del Mar      19
Hotel Camino del Sol      19
Hotel Costa Azul          19
Hotel Encanto Real        19
Hotel Jardines del Rey    19
Hotel Las Estrellas       19
Hotel Los Almendros       19
Hotel Luz de Madrid       19
Hotel Maravilla Real      19
Hotel Mirador Real        19
Hotel Monte Verde         19
Hotel Palacio Imperial    19
Hotel Puerta del Cielo    19
Hotel Rincón Sereno       19
Hotel Sol y Luna          19
Hotel Torre Dorada        19
Hotel Vista Alegre        19
Palacio del Sol           19
Name: id_hotel, dtype: int64

In [40]:
df.groupby("id_hotel")["nombre_hotel"].nunique().sort_values(ascending=False)

id_hotel
2      19
33     19
49     19
48     19
47     19
43     19
3      19
38     19
36     19
35     19
40     19
29     19
27     19
19     19
15     19
14     19
13     19
6      19
28     19
131     1
186     1
181     1
135     1
113     1
128     1
117     1
114     1
103     1
194     1
Name: nombre_hotel, dtype: int64

In [38]:
df.groupby(["id_hotel", "nombre_hotel"]).size().reset_index(name="count").sort_values(by="count", ascending=False)

,id_hotel,nombre_hotel,count
363,114,,537
361,103,,533
365,128,,530
366,131,,524
369,186,,522
...,...,...,...
187,29,Hotel Torre Dorada,17
313,47,Hotel Maravilla Real,16
50,6,Hotel Palacio Imperial,16
171,29,Gran Hotel Madrid,16


In [36]:
df[df["nombre_hotel"] == "Gran Hotel Madrid"][["id_hotel", "nombre_hotel"]].drop_duplicates()


,id_hotel,nombre_hotel
15,47,Gran Hotel Madrid
39,6,Gran Hotel Madrid
45,13,Gran Hotel Madrid
58,15,Gran Hotel Madrid
59,35,Gran Hotel Madrid
77,14,Gran Hotel Madrid
213,49,Gran Hotel Madrid
246,28,Gran Hotel Madrid
267,48,Gran Hotel Madrid
288,38,Gran Hotel Madrid


2️⃣ Analizando clientes con más de una reserva

In [19]:
df.groupby("id_cliente")["id_reserva"].count().reset_index()

,id_cliente,id_reserva
0,00060bb5-2190-44cf-92af-2b029b2d8d3e,1
1,000f196c-13b4-477c-987f-935ad6748255,1
2,0016d018-0481-43bc-ace7-92cf56e8280b,1
3,00193c88-bb57-4910-92f8-98e082f70f54,1
4,00194c55-9178-4224-b94f-d750e101af73,1
...,...,...
14842,fff5a9f1-b46b-43c2-9db1-0641f861c61f,1
14843,fff77a56-464f-437b-8091-c6523c868ca7,1
14844,fffcf520-dab9-41d6-8534-47d767e6d661,1
14845,fffdb9ce-5454-4001-ae80-d928729a6865,1


In [20]:
df_clientes_check = df.groupby("id_cliente")["id_reserva"].count().reset_index()
df_clientes_check = df_clientes_check.sort_values(by="id_reserva", ascending=False)
print(df_clientes_check.head(10))  # Ver los 10 clientes con más reservas

                                 id_cliente  id_reserva
2080   2407853d-23ff-4b7d-abcc-5c619d8c960d           3
6584   7237d336-8fa8-4ab3-b454-0d969f0ecb84           3
14608  fba96362-d117-4f45-b405-f735d813833a           3
8076   8bec1ff1-84d7-4cad-937f-dbae37691569           3
13957  f0c7fbf6-cea3-42fc-a94f-11c69dbf5713           3
10811  ba60cff3-280a-4948-b6da-5e76e57b9d79           3
8430   91cf76ea-170c-459c-82a1-a2dde074ad38           3
2981   3369b2b1-6b9a-4914-b7bb-655dd1d9c90f           3
7024   79ff038f-5dc8-4b72-956b-0e7989d44a06           2
12736  db7b4be7-5857-42c6-b0f0-aec8b5e3ae9a           2


### filtrar los clientes con más de una reserva y ver detalles:

In [21]:
df[df["id_cliente"].isin(df_clientes_check[df_clientes_check["id_reserva"] > 1]["id_cliente"])]


,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,
10,3eca014e-f122-41d7-9dd3-b984b9153d57,dbaadb0e-a0ff-436e-9b10-c697fb90b720,Noé,Lobo,noé.lobo@example.com,False,2025-02-02,2025-03-01,2025-03-02,3,216.57,Hotel Jardines del Rey,3.0,Madrid
24,9b16fc42-a7c6-4db9-98c7-30dbf2bb72b3,614c4625-81ae-4cb8-be62-beef3ba967bb,Francisco Javier,Hierro,francisco javier.hierro@example.com,False,2025-02-02,2025-03-01,2025-03-02,29,437.34,Hotel Sol y Luna,1.0,Madrid
57,9202057b-976a-4e12-980e-d45cac1a2906,3810cb92-a4ee-474d-90d4-c9c9ea04d706,Salud,Rocha,salud.rocha@example.com,False,2025-02-12,2025-03-01,2025-03-02,13,363.46,Hotel Camino del Sol,4.0,Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15093,49c9a352-a260-46fb-9edc-a2c2d05fdfb9,cc50c0c1-a589-44b7-8a41-2c731d033f89,Victoriano,Miralles,victoriano.miralles@example.com,False,2025-02-02,2025-03-01,2025-03-02,36,367.65,Hotel Las Estrellas,3.0,Madrid
15094,f07370e3-feeb-4893-a9ff-4ec4fb518a55,2bf0215f-7f7b-4cfb-9a37-d2022c492410,Anacleto,Toledo,anacleto.toledo@example.com,False,2025-02-07,2025-03-01,2025-03-02,38,144.35,Gran Hotel Madrid,4.0,Madrid
15095,1d4b9293-1733-45ce-8a8e-79d3ac4694eb,4ba34638-c43e-455f-886d-b3e3dd223a07,Máxima,Mayoral,máxima.mayoral@example.com,False,2025-02-08,2025-03-01,2025-03-02,29,463.32,Hotel Jardines del Rey,2.0,Madrid
15096,d6823ca7-e236-48b1-bf68-6b88d9b6534a,a017cb9c-160b-4b90-b4a9-4bdb2dce546a,Rafa,Alcaraz,rafa.alcaraz@example.com,False,2025-02-10,2025-03-01,2025-03-02,15,247.63,Hotel Palacio Imperial,1.0,Madrid


Esto tiene sentido porque, un mismo cliente, ha podido hacer más de una reserva en diferentes hoteles porque en uno mismo no había suficente espacio para todos sus clientes, o también porque tiene cancelación gratuita etc.

3️⃣ Analizando la relación entre fechas y reservas

In [48]:
df["inicio_estancia"].value_counts()

inicio_estancia
2025-03-01    15023
Name: count, dtype: int64

In [50]:
df["final_estancia"].value_counts()

final_estancia
2025-03-02    15023
Name: count, dtype: int64

In [22]:
df["fecha_reserva"].value_counts()

fecha_reserva
2025-02-06    883
2025-02-04    877
2025-02-02    862
2025-02-03    840
2025-02-05    828
2025-02-09    826
2025-02-10    825
2025-02-12    820
2025-02-01    817
2025-02-07    787
2025-02-11    785
2025-02-08    776
Name: count, dtype: int64

In [23]:
df_fechas_check = df.groupby("id_cliente")["fecha_reserva"].nunique().reset_index()
df_fechas_check = df_fechas_check.sort_values(by="fecha_reserva", ascending=False)
print(df_fechas_check[df_fechas_check["fecha_reserva"] > 1])


                                 id_cliente  fecha_reserva
14216  f4d72083-c218-4e30-b4c2-e1d0608ceeca              2
11532  c6660c4e-cec8-4d01-87da-accf30b85362              2
2403   29690e15-e64e-408f-a7a5-a84bb254aef6              2
11942  cd73531b-8ad7-4e27-b247-6525254ae0ea              2
2340   2887dcb4-ce2d-4e01-a5ec-985e59856105              2
...                                     ...            ...
8430   91cf76ea-170c-459c-82a1-a2dde074ad38              2
9528   a42e5ef0-9734-4dcc-bc18-1880513f3c80              2
2679   2dc03d93-8c75-4f34-87bc-91ba86ad31eb              2
13834  ee778940-9785-4ecf-bd1b-fdb137ac826a              2
4890   55070df1-1f08-44ad-a52f-f03594728734              2

[73 rows x 2 columns]


Análisis de duplicados

In [27]:
# Verificar duplicados en el DataFrame
duplicados = df.duplicated()
print(f'Número de filas duplicadas: {duplicados.sum()}')

# Mostrar las filas duplicadas
df[duplicados]

Número de filas duplicadas: 98


,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
15000,9894646a-716a-4a9d-b504-6d05cff706d3,659980f2-c213-4eed-bddf-1bfebc8dc985,Marianela,Pablo,marianela.pablo@example.com,False,2025-02-09,2025-03-01,2025-03-02,29,246.07,Hotel Puerta del Cielo,5.0,Madrid
15001,9754807a-8685-4198-b843-6dc991377377,a8b5e355-d76c-4836-af7d-57021d03228b,Victor,Santamaria,victor.santamaria@example.com,False,2025-02-07,2025-03-01,2025-03-02,49,169.14,Hotel Los Almendros,5.0,Madrid
15002,2a5ea0d1-9584-4458-8a08-e276d7cbbc3a,0f0150df-7805-41dd-84c5-5aadd6816086,Ámbar,Benito,ámbar.benito@example.com,False,2025-02-02,2025-03-01,2025-03-02,19,198.87,Hotel Palacio Imperial,4.0,Madrid
15003,0ba826ae-751c-4cd1-9147-b4c36dd4fc72,c91e350d-1193-41de-834a-1940955e91b6,Tito,Amorós,tito.amorós@example.com,False,2025-02-12,2025-03-01,2025-03-02,14,54.32,Hotel Las Estrellas,1.0,Madrid
15004,1e5d7ead-36e9-4b22-b870-9a122355b763,4ca7383c-3e0c-4403-b0dc-f2a022edb524,Gabriel,Cruz,gabriel.cruz@example.com,False,2025-02-10,2025-03-01,2025-03-02,19,405.23,Hotel Jardines del Rey,5.0,Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15093,49c9a352-a260-46fb-9edc-a2c2d05fdfb9,cc50c0c1-a589-44b7-8a41-2c731d033f89,Victoriano,Miralles,victoriano.miralles@example.com,False,2025-02-02,2025-03-01,2025-03-02,36,367.65,Hotel Las Estrellas,3.0,Madrid
15094,f07370e3-feeb-4893-a9ff-4ec4fb518a55,2bf0215f-7f7b-4cfb-9a37-d2022c492410,Anacleto,Toledo,anacleto.toledo@example.com,False,2025-02-07,2025-03-01,2025-03-02,38,144.35,Gran Hotel Madrid,4.0,Madrid
15095,1d4b9293-1733-45ce-8a8e-79d3ac4694eb,4ba34638-c43e-455f-886d-b3e3dd223a07,Máxima,Mayoral,máxima.mayoral@example.com,False,2025-02-08,2025-03-01,2025-03-02,29,463.32,Hotel Jardines del Rey,2.0,Madrid
15096,d6823ca7-e236-48b1-bf68-6b88d9b6534a,a017cb9c-160b-4b90-b4a9-4bdb2dce546a,Rafa,Alcaraz,rafa.alcaraz@example.com,False,2025-02-10,2025-03-01,2025-03-02,15,247.63,Hotel Palacio Imperial,1.0,Madrid


In [19]:
### limpiamos los duplicados que coinciden en todos sus valores
df = df.drop_duplicates()

Gestión de nulos

In [24]:
def porcentaje_nulos(dataframe):
    porcentaje = dataframe.isnull().mean() * 100
    return porcentaje

# Ver el porcentaje de nulos por columnas
porcentaje_nulos(df)

id_reserva          0.000000
id_cliente          0.000000
nombre              0.000000
apellido            0.000000
mail                0.000000
competencia         0.000000
fecha_reserva      34.256193
inicio_estancia     0.496755
final_estancia      0.496755
id_hotel            0.000000
precio_noche       34.600609
nombre_hotel        0.000000
estrellas          34.256193
ciudad              0.000000
dtype: float64

In [26]:
def porcentaje_nulos_filas(dataframe):
    porcentaje = dataframe.isnull().mean(axis=1) * 100
    return porcentaje

# Ver el porcentaje de nulos por filas
porcentaje_nulos_filas(df)

0        21.428571
1        21.428571
2         0.000000
3         0.000000
4         0.000000
           ...    
15093     0.000000
15094     0.000000
15095     0.000000
15096     0.000000
15097     0.000000
Length: 15098, dtype: float64

4. Validación de Unicidad y Claves Primarias

In [24]:
df['id_reserva'].nunique() == df.shape[0]  # Debe ser True si todos son únicos

True

In [25]:
df['id_hotel'].nunique() == df.shape[0]  # Debe ser True si todos son únicos

False

In [23]:
df['id_cliente'].nunique() == df.shape[0]  # Debe ser True si todos son únicos


False